In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# LOAD DATASET

In [ ]:
salaries_df = pd.read_csv('salary_data_cleaned.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'salary_data_cleaned.csv'

# DATA PREPROCESSING

In [ ]:
salaries_df.head()

In [ ]:
shape = salaries_df.shape
print(f"Records: {shape[0]}, Features: {shape[1]}")

In [ ]:
salaries_df.dtypes


In [ ]:
categorical_columns = salaries_df.select_dtypes(include=['object', 'category']).columns

# Print the number of unique values for each categorical column
for column in categorical_columns:
    print(f'{column} : {len(set(salaries_df[column].unique()))}')

## Feature Visualization

In [ ]:
correlation_matrix = salaries_df.select_dtypes(include=['float64', 'int64']).corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

NameError: name 'salaries_df' is not defined

### Categorical

**These categorical features have over 100 unqiue values so only the first 20 values are plotted.**

In [ ]:
categorical_features_top_20 = ['Job Title', 'Salary Estimate', 'Company Name', 'Location', 'Headquarters', 'Competitors']

plt.figure(figsize=(20, 30))

# Loop through the categorical features and create a subplot for each
for i, feature in enumerate(categorical_features_top_20, start=1):
    top_20 = salaries_df[feature].value_counts().head(20) # Get top 20 values for each feature
    plt.subplot(3, 2, i)
    sns.barplot(x=top_20.values, y=top_20.index, palette='Set3')
    plt.title(f'Distribution of Top 20 {feature}')
    plt.tight_layout()

plt.show()


**These categorical features have less than 100 unqiue values so all their values are plotted.**

In [ ]:
categorical_features = ['Size', 'Type of ownership', 'Sector', 'Revenue', 'job_state', 'Industry']

plt.figure(figsize=(15, 30))

# Loop through the categorical features and create a subplot for each
for i, feature in enumerate(categorical_features, start=1):
    plt.subplot(3, 2, i)  # 3 rows, 2 columns
    # Order the categories by count
    ordered_data = salaries_df[feature].value_counts().index
    sns.countplot(y=feature, data=salaries_df, order=ordered_data, palette='Set3')
    plt.title(f'Distribution of {feature}')
    plt.tight_layout()

plt.show()

### Numerical

**These numerical features are binary.**

In [ ]:
numerical_columns = ['hourly', 'employer_provided', 'same_state']

plt.figure(figsize=(15, 10))

for i, skill in enumerate(numerical_columns, 1):
    plt.subplot(2, 3, i)
    salaries_df[skill].value_counts().plot(kind='bar', color=['lightgreen', 'skyblue'], edgecolor='black')
    plt.xticks(ticks=[1, 0], labels=['Yes', 'No'], rotation=0)
    plt.ylabel('Number of Jobs')

plt.tight_layout()
plt.show()

In [ ]:
tech_skills = ['python_yn', 'R_yn', 'spark', 'aws', 'excel']

plt.figure(figsize=(15, 10))

for i, skill in enumerate(tech_skills, 1):
    plt.subplot(2, 3, i)
    salaries_df[skill].value_counts().plot(kind='bar', color=['lightgreen', 'skyblue'], edgecolor='black')
    plt.title(f'Jobs Requiring {skill.capitalize()}')
    plt.xticks(ticks=[1, 0], labels=['Yes', 'No'], rotation=0)
    plt.ylabel('Number of Jobs')

plt.tight_layout()
plt.show()


**These numerical features were divided into logical bins (i.e rating->1-5 and founded->decades).**

In [ ]:
# Filter out companies with 'Founded' year as -1 (since -1 represents missing or irrelevant data)
df_filtered = salaries_df[salaries_df['Founded'] > 0]

# Create a new column 'Decade' categorizing 'Founded' into decades
# The decade is calculated by dividing the year by 10, converting to integer, and then multiplying by 10
df_filtered['Decade'] = ((df_filtered['Founded'] // 10) * 10).astype(int)

# Count the number of companies founded in each decade
decade_counts = df_filtered['Decade'].value_counts().sort_index()

plt.figure(figsize=(12, 6))
decade_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribution of Companies Founded by Decade')
plt.xlabel('Decade')
plt.ylabel('Number of Companies')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Define the range of bins from the minimum to maximum rating,
# since ratings are between 0 and 5, and include an extra bin for -1 (unrated or missing ratings)
bins = [-1.5] + list(range(int(salaries_df['Rating'].min()), int(salaries_df['Rating'].max()) + 2))

plt.figure(figsize=(10, 6))
plt.hist(salaries_df['Rating'], bins=bins, color="skyblue", edgecolor='black', alpha=0.7)
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Number of Ratings')
plt.xticks(range(int(salaries_df['Rating'].min()), int(salaries_df['Rating'].max()) + 1))  # Setting x-ticks to integer ratings

plt.show()


## Feature Removal

In [ ]:
preprocessed_salaries_df = salaries_df.copy()

In [ ]:
categorical_features_to_remove = [
    "Salary Estimate",
    "Job Description",
    "Headquarters",
    "Size",
    "Industry",
    "Competitors",
    "company_txt",
    "job_state"
]

numerical_features_to_remove = [
    "R_yn",
    "hourly",
    "employer_provided",
    "Founded",
    "Rating",
    "age",
    "same_state"
]

features_to_remove = categorical_features_to_remove + numerical_features_to_remove

In [ ]:
preprocessed_salaries_df = preprocessed_salaries_df.drop(features_to_remove, axis=1)

In [ ]:
print(f"Records: {preprocessed_salaries_df.shape[0]}, Features: {preprocessed_salaries_df.shape[1]}")
preprocessed_salaries_df.head()

## Feature Engineering

### Job Title

In [ ]:
import re

In [ ]:
salaries_df['Job Title'].value_counts()

In [ ]:
def categorize_job_titles(df):
    # Define the regex patterns and their corresponding column names
    regex_to_job_titles = {
        "Data": "data",
        "(Scientist|Scientists|Researcher|Science|Research|R&D)": "scientist",
        "(Engineer|Developer|R&D|Development)": "engineer",
        "(Analyst|Analysis|Analytics)": "analyst",
        "(Senior|Director|VP|Manager|Principal|Sr\.|Sr|Staff|Associate|Jr\.|Jr)": "manager"
    }

    # Initialize a DataFrame to hold the counts
    counts_df = pd.DataFrame(index=salaries_df.index)

    # Initialize columns in counts_df with zeros
    for keyword in regex_to_job_titles.values():
        counts_df[keyword] = 0

    # Loop through each job title and update counts
    for index, row in df.iterrows():
        job_title = row['Job Title']
        for pattern, keyword in regex_to_job_titles.items():
            if re.search(pattern, job_title, re.IGNORECASE):
                counts_df.at[index, keyword] = 1

    return counts_df

In [ ]:
# Test the categorize_job_titles() with a job title from the dataset
job_titles_count_df = categorize_job_titles(preprocessed_salaries_df)

print(f'Job Title: {preprocessed_salaries_df.loc[73, "Job Title"]}\n')
print(f'Word Count: \n{job_titles_count_df.loc[73]}')

In [ ]:
 preprocessed_salaries_df = pd.concat([preprocessed_salaries_df.drop('Job Title', axis=1), job_titles_count_df], axis=1)


In [ ]:
print(f"Records: {preprocessed_salaries_df.shape[0]}, Features: {preprocessed_salaries_df.shape[1]}")
preprocessed_salaries_df.head()

### Location

The commented out code was run locally to obtain the longitude and latitude coordinates of each location. Since it takes a while to get all the locations from the GoogleMap API, the locations were saved in a seperate csv file location_coordinates.csv

In [ ]:
# from dotenv import load_dotenv
# import os
# from googlemaps import Client as GoogleMaps

In [ ]:
# load_dotenv()

# GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

In [ ]:
# gmaps = GoogleMaps(GOOGLE_MAPS_API_KEY)

In [ ]:
# def geocode_location(location):
#     geocode_result = gmaps.geocode(location)
#     if geocode_result:
#         latitude = geocode_result[0]['geometry']['location']['lat']
#         longitude = geocode_result[0]['geometry']['location']['lng']
#         return latitude, longitude
#     else:
#         return None, None

In [ ]:
# # Test the geocode_location() with a location from the dataset
# location_example = salaries_df['Location'][0]
# print(f"Location: {location_example}")

# print(f"Latitude, Longitude: {geocode_location(location_example)}")

In [ ]:
# location_df = salaries_df['Location'].apply(geocode_location).apply(pd.Series)

# location_df.columns = ['latitude', 'longitude']
# print(location_df)

# location_df.to_csv('location_coordinates.csv', index=False) # Save the location coordinates to a CSV file


In [ ]:
location_coordinates_df = pd.read_csv('location_coordinates.csv')
location_coordinates_df.head()

In [ ]:
preprocessed_salaries_df = pd.concat([preprocessed_salaries_df.drop('Location', axis=1), location_coordinates_df], axis=1)

In [ ]:
print(f"Records: {preprocessed_salaries_df.shape[0]}, Features: {preprocessed_salaries_df.shape[1]}")
preprocessed_salaries_df.head()

### Sector

In [ ]:
preprocessed_salaries_df['Sector'] = preprocessed_salaries_df['Sector'].replace('-1', 'Other')

In [ ]:
salaries_df['Sector'].value_counts()

In [ ]:
sector_df = pd.get_dummies(preprocessed_salaries_df['Sector'], prefix='sector')
sector_df.head()

In [ ]:
preprocessed_salaries_df = pd.concat([preprocessed_salaries_df.drop('Sector', axis=1), sector_df], axis=1)

In [ ]:
print(f"Records: {preprocessed_salaries_df.shape[0]}, Features: {preprocessed_salaries_df.shape[1]}")
preprocessed_salaries_df.head()

### Ownership

In [ ]:
salaries_df['Type of ownership'].value_counts()

In [ ]:
def encode_ownership(row):
    ownership_types = {
        "Company - Private": 0,
        "Company - Public": 0,
        "Nonprofit Organization": 0,
        "Subsidiary or Business Segment": 0,
        "Government": 0,
        "Other": 0
    }

    # Check if the row's ownership type matches one of the predefined types
    if row['Type of ownership'] in ownership_types:
        ownership_types[row['Type of ownership']] = 1
    else:
        # If it does not match, categorize as "Other"
        ownership_types["Other"] = 1
    return pd.Series(ownership_types)

In [ ]:
ownership_df = preprocessed_salaries_df.apply(encode_ownership, axis=1)
ownership_df.head()

In [ ]:
preprocessed_salaries_df = pd.concat([preprocessed_salaries_df.drop('Type of ownership', axis=1), ownership_df], axis=1)

In [ ]:
print(f"Records: {preprocessed_salaries_df.shape[0]}, Features: {preprocessed_salaries_df.shape[1]}")
preprocessed_salaries_df.head()

### Revenue

In [ ]:
#Revenue
def encode_revenue(revenue):
    if revenue == "$10+billion (USD)":
        return 10000000000, 10000000000  # Min and Max are both 10 billion

    revenue = revenue.replace('(USD)', '').replace('$', '').strip()
    if 'to' in revenue:
        min_rev, max_rev = revenue.split(' to ')
        min_rev = float(min_rev.replace(' million', '').replace(' billion', '').replace('+', '')) * (1000000 if 'million' in min_rev else 1000000000)
        max_rev = float(max_rev.replace(' million', '').replace(' billion', '').strip()) * (1000000 if 'million' in max_rev else 1000000000)
        return min_rev, max_rev
    elif 'Less than' in revenue:
        return 0, 1000000
    elif 'Unknown' in revenue or '-1' in revenue:
        return None, None
    else:
        # Handles cases with "+", assuming it refers to values at or above the mentioned figure
        if '+' in revenue:
            value = float(revenue.replace(' million', '').replace(' billion', '').replace('+', ''))
            value = value * (1000000 if 'million' in revenue else 1000000000)
            return value, value
        else:
            return None, None

In [ ]:
revenue_df = preprocessed_salaries_df['Revenue'].apply(lambda x: pd.Series(encode_revenue(x)))
revenue_df.columns = ['min_revenue', 'max_revenue']
revenue_df.head()

In [ ]:
preprocessed_salaries_df = pd.concat([preprocessed_salaries_df.drop('Revenue', axis=1), revenue_df], axis=1)

In [ ]:
# filling Na for Unknown instances for 'min_revenue', 'max_revenue'
# Assuming 'df' is your DataFrame
# Calculate the median of the 'min_revenue' column, excluding NaN values
min_revenue_median = preprocessed_salaries_df['min_revenue'].median()

# Fill NaN values in the 'min_revenue' column with the calculated median
preprocessed_salaries_df['min_revenue'] = preprocessed_salaries_df['min_revenue'].fillna(min_revenue_median)

# Calculate the median of the 'max_revenue' column, excluding NaN values
max_revenue_median = preprocessed_salaries_df['max_revenue'].median()

# Fill NaN values in the 'max_revenue' column with the calculated median
preprocessed_salaries_df['max_revenue'] = preprocessed_salaries_df['max_revenue'].fillna(max_revenue_median)

# This updates 'min_revenue' and 'max_revenue' columns in df with NaN values replaced by their respective medians


In [ ]:
print(f"Records: {preprocessed_salaries_df.shape[0]}, Features: {preprocessed_salaries_df.shape[1]}")
preprocessed_salaries_df.head()

In [ ]:
preprocessed_salaries_df = preprocessed_salaries_df.drop(['Company Name'], axis=1)

In [ ]:
preprocessed_salaries_df.dtypes

## Data Normalizing

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
numerical_features = ['latitude', 'longitude', 'min_revenue', 'max_revenue']
binary_features = ['python_yn', 'spark', 'aws', 'excel', 'data', 'scientist', 'engineer', 'analyst', 'manager',
                   'sector_Accounting & Legal', 'sector_Aerospace & Defense', 'sector_Agriculture & Forestry',
                   'sector_Arts, Entertainment & Recreation', 'sector_Biotech & Pharmaceuticals', 'sector_Business Services',
                   'sector_Construction, Repair & Maintenance', 'sector_Consumer Services', 'sector_Education',
                   'sector_Finance', 'sector_Government', 'sector_Health Care', 'sector_Information Technology',
                   'sector_Insurance', 'sector_Manufacturing', 'sector_Media', 'sector_Mining & Metals', 'sector_Non-Profit',
                   'sector_Oil, Gas, Energy & Utilities', 'sector_Other', 'sector_Real Estate', 'sector_Retail',
                   'sector_Telecommunications', 'sector_Transportation & Logistics', 'sector_Travel & Tourism',
                   'Company - Private', 'Company - Public', 'Nonprofit Organization', 'Subsidiary or Business Segment',
                   'Government', 'Other']


In [ ]:
scaler = MinMaxScaler()

# Scale numerical features
preprocessed_salaries_df[numerical_features] = scaler.fit_transform(preprocessed_salaries_df[numerical_features])


In [ ]:
preprocessed_salaries_df

In [ ]:
na_count_column = preprocessed_salaries_df['min_revenue'].isna().sum()

print(f"Number of NaNs in the column: {na_count_column}")

#KNN model

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming preprocessed_salaries_df is your DataFrame
X = preprocessed_salaries_df.drop(['min_salary', 'max_salary', 'avg_salary'], axis=1)
y = preprocessed_salaries_df[['min_salary', 'max_salary', 'avg_salary']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a range of k values to try
k_values = range(1, 30)
best_rmse = np.inf
best_k = 0

# Loop over each k value, train the model, and compute the RMSE on the testing set
for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)

    # Make predictions on the testing set
    test_predictions = knn.predict(X_test)

    # Calculate average RMSE for the testing set
    avg_rmse_test = np.mean([
        mean_squared_error(y_test['min_salary'], test_predictions[:, 0], squared=False),
        mean_squared_error(y_test['max_salary'], test_predictions[:, 1], squared=False),
        mean_squared_error(y_test['avg_salary'], test_predictions[:, 2], squared=False)
    ])

    # Update the best_k if the current average RMSE is lower
    if avg_rmse_test < best_rmse:
        best_rmse = avg_rmse_test
        best_k = k

# Re-train the model with the best_k found
knn_best = KNeighborsRegressor(n_neighbors=best_k)
knn_best.fit(X_train, y_train)

# Make predictions with the best model for both training and testing sets
train_predictions_best = knn_best.predict(X_train)
test_predictions_best = knn_best.predict(X_test)

# Calculate RMSE for the training and testing sets with the best model
# Training set
rmse_train_min = mean_squared_error(y_train['min_salary'], train_predictions_best[:, 0], squared=False)
rmse_train_max = mean_squared_error(y_train['max_salary'], train_predictions_best[:, 1], squared=False)
rmse_train_avg = mean_squared_error(y_train['avg_salary'], train_predictions_best[:, 2], squared=False)

# Testing set
rmse_test_min = mean_squared_error(y_test['min_salary'], test_predictions_best[:, 0], squared=False)
rmse_test_max = mean_squared_error(y_test['max_salary'], test_predictions_best[:, 1], squared=False)
rmse_test_avg = mean_squared_error(y_test['avg_salary'], test_predictions_best[:, 2], squared=False)

# Print out the best K value and RMSE for both sets
print(f"Best K Value: {best_k}")
print("\nTraining Set RMSE with Best K:")
print(f"Min Salary: {rmse_train_min}")
print(f"Max Salary: {rmse_train_max}")
print(f"Avg Salary: {rmse_train_avg}")

print("\nTesting Set RMSE with Best K:")
print(f"Min Salary: {rmse_test_min}")
print(f"Max Salary: {rmse_test_max}")
print(f"Avg Salary: {rmse_test_avg}")
